In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/opt/psegs')

from psegs.datasets import ios_lidar

base_dir = '/outer_root/home/au/lidarphone_scans/2021_06_27_12_37_38'
# base_dir = '/outer_root/home/au/lidarphone_scans/landscape_home_button_right_07_09_49'

from oarphpy import util as oputil
json_paths = oputil.all_files_recursive(base_dir, pattern='frame*.json')
json_paths = sorted(json_paths)
cis = [ios_lidar.threeDScannerApp_create_camera_image(p) for p in json_paths]

print(len(cis))
cis[0]
    

149


CameraImage(sensor_name='camera_front', image_jpeg=bytearray(b''), image_png=bytearray(b''), image_factory=CloudpickeledCallable(_func_pyclass=psegs.datasets.ios_lidar.<lambda>), width=1920, height=1440, timestamp=853720759521708, ego_pose=Transform(rotation=array([[-0.16402921,  0.36367464, -0.91697067],
       [-0.04168281,  0.92617565,  0.37478161],
       [ 0.98557436,  0.09969707, -0.13676088]]), translation=array([[-2.92258954],
       [ 0.93101001],
       [ 0.19710636]]), src_frame='ego', dest_frame='world'), ego_to_sensor=Transform(rotation=array([[-1,  0,  0],
       [ 0, -1,  0],
       [ 0,  0, -1]]), translation=array([[0.],
       [0.],
       [0.]]), src_frame='camera_front', dest_frame='ego'), K=array([[1.44565613e+03, 0.00000000e+00, 9.69324890e+02],
       [0.00000000e+00, 1.44565613e+03, 6.90660400e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), extra={'threeDScannerApp.cameraGrain': '0', 'threeDScannerApp.time': '853720.7595217085', 'threeDScannerAp

In [ ]:
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=False)         # initiate notebook for offline plot

In [ ]:
import os
import numpy as np
import open3d as o3d
mesh = o3d.io.read_triangle_mesh(os.path.join(base_dir, 'export_refined.obj'))
vertices = np.asarray(mesh.vertices)
print(vertices.shape)

import numpy as np
v_sub = vertices[np.random.choice(vertices.shape[0], 40000, replace=False)]
print(v_sub.shape)

In [ ]:
import plotly
import plotly.graph_objects as go

plots = [ci.to_plotly_world_frame_3d() for ci in cis]

import pandas as pd
cloud_df = pd.DataFrame(v_sub, columns=['x', 'y', 'z'])
from psegs.util.plotting import rgb_for_distance
cloud_df['color'] = [
  rgb_for_distance(np.linalg.norm(pt), period_meters=1.)
  for pt in cloud_df[['x', 'y', 'z']].values
]
scatter = go.Scatter3d(
                x=cloud_df['x'], y=cloud_df['y'], z=cloud_df['z'],
                mode='markers',
                marker=dict(size=2, color=cloud_df['color'], opacity=0.5),)

plots.append(scatter)


fig = go.Figure(data=plots)

fig.update_layout(
  width=1000, height=700,
  scene_aspectmode='data')
  # scene_camera=dict(
  #   up=dict(x=0, y=0, z=1),
  #   eye=dict(x=0, y=0, z=0),
  #   center=dict(x=1, y=0, z=0),
  # ))
    
iplot(fig)
    
    
# plot_str = plotly.offline.plot(fig, output_type='div')

# html += '<br/><br/>' + plot_str




In [ ]:
# # https://chart-studio.plotly.com/~empet/15040/plotly-mesh3d-from-a-wavefront-obj-f/#/


# def obj_data_to_mesh3d(odata):
#     # odata is the string read from an obj file
#     vertices = []
#     faces = []
#     lines = odata.splitlines()   
   
#     for line in lines:
#         slist = line.split()
#         if slist:
#             if slist[0] == 'v':
#                 vertex = np.array(slist[1:], dtype=float)
#                 vertices.append(vertex)
#             elif slist[0] == 'f':
#                 face = []
#                 for k in range(1, len(slist)):
#                     face.append([int(s) for s in slist[k].replace('//','/').split('/')])
#                 if len(face) > 3: # triangulate the n-polyonal face, n>3
#                     faces.extend([[face[0][0]-1, face[k][0]-1, face[k+1][0]-1] for k in range(1, len(face)-1)])
#                 else:    
#                     faces.append([face[j][0]-1 for j in range(len(face))])
#             else: pass
    
    
#     return np.array(vertices), np.array(faces)

# import os
# with open(os.path.join(base_dir, 'export_refined.obj'), 'rb') as f:
#     obj_data = f.read().decode('utf-8')
# vertices, faces = obj_data_to_mesh3d(obj_data)
# print('vertices.shape', vertices.shape)
# print('faces.shape', faces.shape)


# x, y, z = vertices[:,:3].T
# I, J, K = faces.T

# mesh = go.Mesh3d(
#             x=-x,
#             y=-y,
#             z=z,
# #             vertexcolor=vertices[:, 3:], #the color codes must be triplets of floats  in [0,1]!!                      
#             i=I,
#             j=J,
#             k=K,
#             name='',
#             showscale=False)

# layout = go.Layout(width=900,
#                    height=800,
#                    scene=dict(xaxis=dict(visible=False),
#                               yaxis=dict(visible=False),  
#                               zaxis=dict(visible=False), 
#                               aspectratio=dict(x=1.5,
#                                                y=0.9,
#                                                z=0.5
#                                          ),
#                               camera=dict(eye=dict(x=1., y=1., z=0.5)),
#                         ),
#                   ) 

# fig = go.Figure(data=[mesh], layout=layout)
# iplot(fig)

In [ ]:
# import open3d as o3d
# mesh = o3d.io.read_triangle_mesh(os.path.join(base_dir, 'export_refined.obj'))
# vertices = np.asarray(mesh.vertices)
# print(vertices.shape)

# import numpy as np
# v_sub = vertices[np.random.choice(vertices.shape[0], 10000, replace=False)]
# print(v_sub.shape)

In [ ]:
cloud_raw = vertices

import imageio
writer = imageio.get_writer('/outer_root/home/au/lidarphone_scans/test_video.mp4', fps=5)

for i in range(len(cis)):
    cloud_ego = cis[i].ego_pose.get_inverse().apply(cloud_raw).T
    cloud_ego[:, 0] *= -1

    from psegs import datum
    pc = datum.PointCloud(cloud=cloud_ego)

    debug = cis[i].get_debug_image(clouds=[pc], period_meters=0.1)
    writer.append_data(debug)
    print(i)
writer.close()

#     from io import BytesIO
#     import IPython.display
#     import numpy as np
#     import PIL.Image
#     def showarray(a, fmt='png'):
#         a = np.uint8(a)
#         f = BytesIO()
#         PIL.Image.fromarray(a).save(f, fmt)
#         IPython.display.display(IPython.display.Image(data=f.getvalue()))

#     showarray(debug)

In [5]:
import sys
sys.path.append('/opt/psegs')

import os

ROOT = '/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/'

for d in os.listdir(ROOT):
    if '.DS_Store' in d:
        continue
    base_dir = os.path.join(ROOT, d)
    print(base_dir)
    

    from psegs.datasets import ios_lidar


    from oarphpy import util as oputil
    json_paths = oputil.all_files_recursive(base_dir, pattern='frame*.json')
    json_paths = sorted(json_paths)
    
    try:
        cis = [ios_lidar.threeDScannerApp_create_camera_image(p) for p in json_paths]
    except AssertionError as e:
        continue

    print(len(cis))
    
    import os
    import numpy as np
    import open3d as o3d
    mesh = o3d.io.read_triangle_mesh(os.path.join(base_dir, 'export.obj'))
    vertices = np.asarray(mesh.vertices)
    print(vertices.shape)

    cloud_raw = vertices

    outpath = os.path.join(ROOT, d + '.mp4')
    
    import imageio
    writer = imageio.get_writer(outpath, fps=5)

    for i in range(len(cis)):
        cloud_ego = cis[i].ego_pose.get_inverse().apply(cloud_raw).T
        cloud_ego[:, 0] *= -1

        from psegs import datum
        pc = datum.PointCloud(cloud=cloud_ego)

        debug = cis[i].get_debug_image(clouds=[pc], period_meters=0.1)
        writer.append_data(debug)
        print(i)
    writer.close()
    
    print('done', outpath)


/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_31_01
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_33_35
24
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(39153, 3)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_33_35.mp4
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_33_35.mp4
0
[Open3D WARNING] Unable to load file /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_33_35.mp4/export.obj with ASSIMP
(0, 3)
done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_33_35.mp4.mp4
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_09_16_33_55
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_0

54
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(325597, 3)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_12_23_27_18.mp4
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_12_23_30_41
54
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(21839, 3)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_12_23_30_41.mp4
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_14_14_06_28
31
[Open3D INFO] Skipping non-tri

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_17_15_41_44.mp4
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_23_14_54_38
35
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(2508145, 3)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_23_14_54_38.mp4
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_23_14_55_14
37
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(2898864, 3)
0
1
2
3
4
5
6
7
8
9
1

102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_12_26_57.mp4
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_12_29_50
7
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(4612060, 3)
0
1
2
3
4
5
6
done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_12_29_50.mp4
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_12_31_43
7
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(317320, 3)
0
1
2
3
4
5
6
done /outer_root/media/red14000/Pictures_

25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_12_59_00.mp4
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_06_27_13_00_15
235
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(4698429, 3)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_01_12_09_24.mp4
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_01_13_05_54
60
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(1460044, 3)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_01_13_05_54.mp4
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_01_13_09_56
111
[Open3D INFO] Skipping non-tri

155
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(1933401, 3)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_24_15_54_17.mp4
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_07_25_23_58_28
40
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(10102, 3)
0
1
2
3
4


322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
done /outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_08_05_14_17_29.mp4
/outer_root/media/red14000/Pictures_and_Docs/lidarphone_lidar_scans/2021_08_05_14_28_12
307
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Ski